In [ ]:
# Importar librerías y módulos necesarios
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Añadir el directorio raíz del proyecto al sys.path
project_root = os.path.abspath('..')
sys.path.append(project_root)

from src.sound_signal import Signal
from src.composite_signal import CompositeSignal
from src.rhythm_signal import RhythmSignal
from src.utils import (
    calculate_note_frequencies,
    plot_spectrogram,
    plot_mfcc,
    freq_to_note_name,
    normalize_signal
)

import pandas as pd
import seaborn as sns
import random

### Definicion de Parametros y Directorios

In [ ]:
# Parámetros
SAMPLE_RATE = 44100
DURATION_SEG = 5.0
AMPLITUDE = 1.0
PHASE = 0.0
OCTAVE_RANGE = (0, 8)  # Cubrimos las octavas de 0 a 8
REF_FREQ = 440.0

# Tonalidades disponibles
TONALIDADES = [
    'C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'Ab', 'A', 'Bb', 'B'
]

# Directorio base de datos
data_dir = os.path.join(project_root, 'data')

# Directorio de salida para tonos simples
output_dir_tones = os.path.join(data_dir, 'tones')
os.makedirs(output_dir_tones, exist_ok=True)

# Directorio de salida para acordes
output_dir_chords = os.path.join(data_dir, 'chords')
os.makedirs(output_dir_chords, exist_ok=True)

# Directorio de salida para melodías
output_dir_melodies = os.path.join(data_dir, 'melodies')
os.makedirs(output_dir_melodies, exist_ok=True)

# Directorio de salida para señales superpuestas
output_dir_superposed = os.path.join(data_dir, 'superposed')
os.makedirs(output_dir_superposed, exist_ok=True)

### Calculo de Frecuencias

In [ ]:
# Calcular las frecuencias de las notas para todas las octavas
note_freqs = calculate_note_frequencies(octave_range=OCTAVE_RANGE, ref_freq=REF_FREQ)

# Mostrar algunas frecuencias de ejemplo de diferentes octavas
print("Algunas frecuencias de ejemplo de diferentes octavas:")
examples = [
    ('C0', 'C4', 'C8'),
    ('E0', 'E4', 'E8'),
    ('G0', 'G4', 'G8'),
    ('A0', 'A4', 'A8'),
]
for notes in examples:
    for note in notes:
        freq = note_freqs.get(note)
        if freq:
            print(f"{note}: {freq:.3f} Hz")
        else:
            print(f"{note}: Nota no encontrada.")
    print("---")


### Definicion de Escalas y Modos

In [ ]:
# Definir intervalos de escalas y modos
SCALES = {
    'major': [0, 2, 4, 5, 7, 9, 11],
    'minor_natural': [0, 2, 3, 5, 7, 8, 10],
    'dorian': [0, 2, 3, 5, 7, 9, 10],
    'phrygian': [0, 1, 3, 5, 7, 8, 10],
    'lydian': [0, 2, 4, 6, 7, 9, 11],
    'mixolydian': [0, 2, 4, 5, 7, 9, 10],
    'locrian': [0, 1, 3, 5, 6, 8, 10],
    'pentatonic_major': [0, 2, 4, 7, 9],
    'pentatonic_minor': [0, 3, 5, 7, 10],
    'blues': [0, 3, 5, 6, 7, 10],
}

### Funciones para Generar Senales

In [ ]:
def get_scale_notes(tonic, scale_name, note_frequencies):
    """
    Genera una lista de notas pertenecientes a la escala dada a partir de la tónica.
    """
    intervals = SCALES.get(scale_name)
    if not intervals:
        print(f"Escala {scale_name} no definida.")
        return []
    
    # Generar las notas de la escala en todas las octavas
    scale_notes = []
    for octave in range(OCTAVE_RANGE[0], OCTAVE_RANGE[1] + 1):
        tonic_note = f"{tonic}{octave}"
        if tonic_note not in note_frequencies:
            continue
        for interval in intervals:
            note_semitone = (NOTE_SEMITONES[tonic] + interval) % 12
            note_name = SEMITONE_NOTES[note_semitone]
            note = f"{note_name}{octave}"
            if note in note_frequencies:
                scale_notes.append(note)
    return scale_notes

# Mapas auxiliares para calcular las notas
SEMITONE_NOTES = {
    0: 'C', 1: 'C#', 2: 'D', 3: 'Eb', 4: 'E', 5: 'F', 6: 'F#', 7: 'G',
    8: 'Ab', 9: 'A', 10: 'Bb', 11: 'B'
}
NOTE_SEMITONES = {v: k for k, v in SEMITONE_NOTES.items()}


In [ ]:
def generate_and_save_tone(note: str, note_frequencies: dict,
                           amplitude: float = AMPLITUDE, phase: float = PHASE,
                           duration: float = DURATION_SEG, sample_rate: int = SAMPLE_RATE,
                           output_dir: str = output_dir_tones) -> None:
    """
    Genera un tono simple para la nota dada y guarda el archivo WAV.
    """
    freq = note_frequencies.get(note)
    if freq:
        signal = Signal(amplitude=amplitude, freq=freq, phase=phase,
                        sample_rate=sample_rate, duration_seg=duration)
        signal.generate_signal()
        filename = os.path.join(output_dir, f'{note}.wav')
        signal.save_wav(filename=filename)
        print(f'Tono {note} generado y guardado en {filename}')
    else:
        print(f'Frecuencia para la nota {note} no encontrada.')
        
def generate_and_save_chord(chord_name: str, notes: list, note_frequencies: dict,
                            amplitude: float = AMPLITUDE, phase: float = PHASE,
                            duration: float = DURATION_SEG, sample_rate: int = SAMPLE_RATE,
                            output_dir: str = output_dir_chords) -> None:
    """
    Genera un acorde a partir de una lista de notas y guarda el archivo WAV.
    """
    composite = CompositeSignal(sample_rate=sample_rate, duration_seg=duration)
    for note in notes:
        freq = note_frequencies.get(note)
        if freq:
            composite.add_component(amplitude=amplitude, freq=freq, phase=phase)
        else:
            print(f'Frecuencia para la nota {note} no encontrada.')
    composite.build_signal()
    
    # Guardar el archivo WAV
    filename = os.path.join(output_dir, f'{chord_name}.wav')
    composite.save_wav(filename=filename)
    
    # Generar espectrograma y MFCC
    spectrogram_path = filename.replace('.wav', '_spectrogram.png')
    mfcc_path = filename.replace('.wav', '_mfcc.png')
    plot_spectrogram(composite.signal, composite.sample_rate, save_path=spectrogram_path)
    plot_mfcc(composite.signal, composite.sample_rate, save_path=mfcc_path)
    
    print(f'Acorde {chord_name} generado y guardado en {filename}')

def generate_diatonic_chords(tonic: str, scale_name: str, note_frequencies: dict):
    """
    Genera acordes diatónicos para la tonalidad y escala dada.
    """
    scale_notes = get_scale_notes(tonic, scale_name, note_frequencies)
    if not scale_notes:
        return {}
    
    chords = {}
    num_notes_in_scale = len(scale_notes)
    for i in range(num_notes_in_scale):
        chord_notes = []
        for j in [0, 2, 4]:  # Triadas (I, III, V)
            note_index = (i + j)
            if note_index < num_notes_in_scale:
                chord_notes.append(scale_notes[note_index])
        chord_name = f"{tonic}_{scale_name}_Chord_{i+1}"
        chords[chord_name] = chord_notes
    return chords


In [ ]:
def generate_and_save_melody(tonic: str, scale_name: str, note_frequencies: dict,
                             num_notes: int = 16, amplitude: float = AMPLITUDE,
                             phase: float = PHASE, duration: float = DURATION_SEG,
                             sample_rate: int = SAMPLE_RATE, output_dir: str = output_dir_melodies) -> None:
    """
    Genera una melodía aleatoria basada en la escala y tonalidad dadas.
    """
    scale_notes = get_scale_notes(tonic, scale_name, note_frequencies)
    if not scale_notes:
        return
    
    # Seleccionar un rango de octavas para la melodía
    available_octaves = range(OCTAVE_RANGE[0], OCTAVE_RANGE[1] + 1)
    melody_octaves = random.sample(available_octaves, k=min(3, len(available_octaves)))
    melody_notes = [note for note in scale_notes if int(note[-1]) in melody_octaves]
    
    unit_time = duration / num_notes
    melody = RhythmSignal(sample_rate=sample_rate, duration_seg=duration, unit_time=unit_time)
    
    for i in range(num_notes):
        note = random.choice(melody_notes)
        freq = note_frequencies.get(note)
        if freq:
            segment_components = [{'amplitude': amplitude, 'freq': freq, 'phase': phase}]
            melody.add_segment(segment_components, label=note)
        else:
            print(f'Frecuencia para la nota {note} no encontrada.')
    
    melody.build_signal()
    
    # Guardar el archivo WAV
    filename = os.path.join(output_dir, f'melody_{tonic}_{scale_name}_{random.randint(0, 9999)}.wav')
    melody.save_wav(filename=filename)
    
    # Generar espectrograma y MFCC
    spectrogram_path = filename.replace('.wav', '_spectrogram.png')
    mfcc_path = filename.replace('.wav', '_mfcc.png')
    plot_spectrogram(melody.signal, melody.sample_rate, save_path=spectrogram_path)
    plot_mfcc(melody.signal, melody.sample_rate, save_path=mfcc_path)
    
    print(f'Melodía generada en tonalidad {tonic} {scale_name} y guardada en {filename}')


In [ ]:
def generate_and_save_superposed_signal(melody_filename: str, chord_filename: str,
                                        output_dir: str = output_dir_superposed) -> None:
    """
    Superpone una melodía y un acorde adicional, y guarda el archivo WAV, espectrograma y MFCC.
    """
    melody_path = os.path.join(output_dir_melodies, melody_filename)
    chord_path = os.path.join(output_dir_chords, chord_filename)
    
    # Verificar si los archivos existen
    if not os.path.isfile(melody_path):
        print(f'Melodía {melody_filename} no encontrada.')
        return
    if not os.path.isfile(chord_path):
        print(f'Acorde {chord_filename} no encontrado.')
        return
    
    # Cargar las señales
    melody_signal = RhythmSignal(sample_rate=SAMPLE_RATE, duration_seg=DURATION_SEG)
    melody_signal.load_wav(melody_path)
    
    chord_signal = CompositeSignal(sample_rate=SAMPLE_RATE, duration_seg=DURATION_SEG)
    chord_signal.load_wav(chord_path)
    
    # Crear la señal superpuesta
    superposed = CompositeSignal(sample_rate=SAMPLE_RATE, duration_seg=DURATION_SEG)
    superposed.add_signal(melody_signal)
    superposed.add_signal(chord_signal)
    superposed.build_signal()
    
    # Guardar el archivo WAV superpuesto
    filename_superposed = os.path.join(
        output_dir,
        f'superposed_{melody_filename[:-4]}_{chord_filename[:-4]}.wav'
    )
    superposed.save_wav(filename=filename_superposed)
    
    # Generar espectrograma y MFCC
    spectrogram_path_superposed = filename_superposed.replace('.wav', '_spectrogram.png')
    mfcc_path_superposed = filename_superposed.replace('.wav', '_mfcc.png')
    plot_spectrogram(superposed.signal, superposed.sample_rate, save_path=spectrogram_path_superposed)
    plot_mfcc(superposed.signal, superposed.sample_rate, save_path=mfcc_path_superposed)
    
    print(f'Señal superpuesta generada y guardada en {filename_superposed}')


In [ ]:
def generate_dataset(note_frequencies: dict):
    """
    Genera tonos, acordes, melodías y señales superpuestas para el conjunto de datos.
    """
    # Generar tonos
    print("Generando tonos simples...")
    for note in note_frequencies.keys():
        generate_and_save_tone(note=note, note_frequencies=note_frequencies)
    
    # Generar acordes
    tonalidades = ['C', 'G', 'D', 'A', 'E', 'F', 'Bb', 'Eb']
    print("Generando acordes diatónicos...")
    for tonic in tonalidades:
        chords = generate_diatonic_chords(tonic=tonic, scale_name='major', note_frequencies=note_frequencies)
        for chord_name, notes in chords.items():
            generate_and_save_chord(chord_name=chord_name, notes=notes, note_frequencies=note_frequencies)
    
    # Generar melodías
    escalas = ['major', 'minor_natural', 'dorian', 'mixolydian', 'lydian']
    print("Generando melodías...")
    for tonic in tonalidades:
        for scale_name in escalas:
            for _ in range(5):  # Generar 5 melodías por tonalidad y escala
                generate_and_save_melody(tonic=tonic, scale_name=scale_name, note_frequencies=note_frequencies)
    
    # Generar señales superpuestas
    print("Generando señales superpuestas...")
    melody_files = os.listdir(output_dir_melodies)
    chord_files = os.listdir(output_dir_chords)
    for i in range(50):  # Generar 50 señales superpuestas
        melody_file = random.choice(melody_files)
        chord_file = random.choice(chord_files)
        generate_and_save_superposed_signal(melody_filename=melody_file, chord_filename=chord_file)


### Generacion de Conjunto de Datos

In [ ]:
# Ejecutar la generación del conjunto de datos
generate_dataset(note_freqs)
